<a href="https://colab.research.google.com/github/dineshkumarkarimajji/INVOICE-PRO/blob/main/InvoicePro_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import json
import os
from datetime import datetime

#Product
class Product:
  def __init__(self,id, name, price, stock):
    self.id=id
    self.name = name
    self.price = price
    self.stock = stock

  def update_stock(self, new_stock):
    self.stock += new_stock #Increasing stock

  def to_dict(self):
    return {
      "id": self.id,
      "name": self.name,
      "price": self.price,
      "stock": self.stock
    }
  @staticmethod
  def from_dict(dict):
    return Product(dict["id"], dict["name"], dict["price"], dict["stock"])

#product done#

#----InvoiceItem---#
class InvoiceItem:
  def __init__(self, product, quantity):
    self.product = product
    self.quantity = quantity
    self.total_price = product.price * quantity
# invoiceItem done #

#----Invoice--#
class Invoice:
  TAX_RATE = 0.13
  def __init__(self, customer_name: str):
    self.customer_name = customer_name
    self.items = []
    self.subtotal=0
    self.tax=0
    self.total=0

  def add_item(self, product, quantity):
    if quantity > product.stock:
      print(f"Not enough stock for {product.name}. Available stock: {product.stock}")
      return
    self.items.append(InvoiceItem(product, quantity))
    product.update_stock(-quantity)

  def calculate_totals(self):
    self.subtotal = sum(item.total_price for item in self.items)
    self.tax = self.subtotal * self.TAX_RATE
    self.total = self.subtotal + self.tax

  def print_invoice(self):
    self.calculate_totals()
    print("\n" + "=" * 50)
    print(" " * 15 + "INVOICE")
    print("=" * 50)
    print(f"Customer: {self.customer_name}")
    print(f"Date    : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("-" * 50)
    print(f"{'ID':<5}{'Product':<20}{'Qty':<5}{'Price':<8}{'Total':<8}")
    print("-" * 50)
    for item in self.items:
      print(f"{item.product.id:<5}{item.product.name:<20}{item.quantity:<5}${item.product.price:<8}${item.total_price:<8}")
    print("-" * 50)
    print(f"{'Subtotal':<45}${self.subtotal:<8}")
    print(f"{'Tax':<45}${self.tax:<8}")
    print(f"{'Total':<45}${self.total:<8}")
    print("-" * 50)

  def save_invoice(self):
    self.calculate_totals()
    filename = f"invoices/{self.customer_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
    os.makedirs("invoices", exist_ok=True)
    with open(filename, "w") as f:
      f.write("INVOICE\n")
      f.write("=" * 50 + "\n")
      f.write(f"Customer: {self.customer_name}\n")
      f.write(f"Date    : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
      f.write("-" * 50 + "\n")
      for item in self.items:
        f.write(f"{item.product.id} {item.product.name} x{item.quantity} "
          f"= {item.total_price:.2f}\n")
      f.write("-" * 50 + "\n")
      f.write(f"Subtotal: {self.subtotal:.2f}\n")
      f.write(f"Tax (10%): {self.tax:.2f}\n")
      f.write(f"TOTAL: {self.total:.2f}\n")
    print(f"✅ Invoice saved as {filename}")

#---invoice done--#

#---Inventory manager--#
class InventoryManager :
  def __init__(self,filepath="products.json"):
    self.filepath=filepath
    self.products=[]
    self.load_products()

  def add_product(self,product):
    self.products.append(product)
    self.save_products()


  def update_Product(self,id,name,price,stock):
    prod=self.find_product(id)
    if not prod:
      print("Product not found")
      return
    if name: prod.name=name
    if price: prod.price=price
    if stock: prod.stock=stock
    self.save_products()

  def find_product(self,id):
    for prod in self.products:
      if prod.id==id:
        return prod
  def view_products(self):
    print("\nAvailable products:")
    print(f"{'ID':<5}{'Product':<20}{'Price':<8}{'Stock':<8}")
    for prod in self.products:
        print(f"{prod.id:<5}{prod.name:<20}${prod.price:<8}{prod.stock:<8}")

  def save_products(self):
    with open(self.filepath,"w") as f:
      json.dump([p.to_dict() for p in self.products],f, indent=2)

  def load_products(self):
    if os.path.exists(self.filepath):
      with open(self.filepath, 'r') as f:
        data=json.load(f)
        self.products=[Product.from_dict(p) for p in data]
#Inventory manager done#

#----main menu-------#
def main():
  inventory_manager=InventoryManager()

  while True:
    print("\n📌 InvoicePro Menu:")
    print("1. View Products")
    print("2. Add Product")
    print("3. Update Product")
    print("4. Create Invoice")
    print("5. Exit")
    choice = input("Enter choice: ").strip()

    if choice=="1":
      inventory_manager.view_products()

    elif choice == "2":
      id = input("Enter product ID: ")
      name = input("Enter product name: ")
      price = float(input("Enter product price: "))
      stock = int(input("Enter product stock: "))
      inventory_manager.add_product(Product(id, name, price, stock))

    elif choice == "3":
      id = input("Enter product ID: ")
      name = input("Enter product name: ")
      price = input("Enter product price: ")
      price = float(price) if price.strip() else None
      stock = input("Enter product stock: ")
      stock = int(stock) if stock.strip() else None
      inventory_manager.update_Product(id, name, price, stock)

    elif choice=="4":
      customer_name=input("Enter customer name: ")
      invoice=Invoice(customer_name)
      while True:
        inventory_manager.view_products()
        id=input("Enter product ID to add(or 'done')").strip()
        if id.lower()=="done":
          break
        product=inventory_manager.find_product(id)
        if not product:
          print("Product not found")
          continue
        quantity=int(input("Enter quantity: "))
        invoice.add_item(product,quantity)
      invoice.print_invoice()
      invoice.save_invoice()
    elif choice=="5":
      print("Exiting...")
      break
    else:
      print("Invalid choice. Please try again.")

  print("Thank you for using InvoicePro!")

if __name__=="__main__":
  main()



📌 InvoicePro Menu:
1. View Products
2. Add Product
3. Update Product
4. Create Invoice
5. Exit
Enter choice: 4
Enter customer name: dinesh

Available products:
ID   Product             Price   Stock   
P001 Apple               $10.0    50      
P002 Mango               $5.0     100     
Enter product ID to add(or 'done')P001
Enter quantity: 10

Available products:
ID   Product             Price   Stock   
P001 Apple               $10.0    40      
P002 Mango               $5.0     100     
Enter product ID to add(or 'done')done

               INVOICE
Customer: dinesh
Date    : 2025-08-13 16:11:51
--------------------------------------------------
ID   Product             Qty  Price   Total   
--------------------------------------------------
P001 Apple               10   $10.0    $100.0   
--------------------------------------------------
Subtotal                                     $100.0   
Tax                                          $13.0    
Total                              